# 13. 난 스케치를 할 테니 너는 채색을 하거라
## 13-2. 조건 없는 생성모델(Unconditional Generative Model), GAN

잘 학습된 GAN을 이용해 실제 이미지를 생성할 경우, 자신이 원하는 종류의 이미지를 바로 생성해내지 못한다.
- 일반적으로 GAN과 같은 unconditional generative model은 **생성하고자 하는 데이터에 대한 제어가 힘들다**

[GAN의 문제점](https://go-hard.tistory.com/38)


## 13-3. 조건 있는 생성모델(Conditional Generative Model), cGAN

> GAN 복습

- GAN 구조는 Generator 및 Discriminator라고 하는 두 신경망이 minmax game으로 경쟁하며 발전함
- 아래의 수식으로 표현할 수 있음

$$ min_{G}max_{D}V (D,G) = E_{x~P_{data}}[logD(x)] + E_{z~p_{x}(z)}[log(1 - D(G(z)))]} $$

`z`는 임의 노이즈, `D`는 Discriminator, `G`는 Generator를 의미</br>

최종적으로 G는 z를 입력받아 생성한 데이터 G(z)를 D가 진짜 데이터라고 예측할 만큼 진짜 같은 가짜 데이터를 만들도록 학습한다는 뜻.</br>

cGAN의 목적 함수</br>

$$ min_{G}max_{D}V (D,G) = E_{x~P_{data}}[logD(x|y)] + E_{z~p_{x}(z)}[log(1 - D(G(z|y)))]} $$

y의 의미: G와 D의 입력에 특정 조건을 나타내는 정보</br>


**그림으로 이해하기**</br>

GAN의 학습 과정</br>
![GAN feed forward](../images/lec13/1.png)</br>
- **Generator**
    노이즈 z(파란색)가 입력, 특징 representation(검정색)으로 변환, 가짜 데이터 G(z) (빨간색)을 생성
- **Discriminator**
    실제 데이터 x와 Generator가 생성한 가짜 데이터 G(z)를 각각 입력, D(x), D(G(z)) (보라색)을 계산
    하여 진짜와 가짜를 식별

cGAN의 학습 과정</br>
![cGAN feed forward](../images/lec13/2.png)</br>
GAN과 다르게 y라는 정보가 함께 입력되는 차이점이 있다.
- **Generator**
    노이즈 z(파란색)와 추가정보 y(녹색)을 함께 입력받아 Generator 내부에서 결합, 결합된 정보는 respresentation(검정색)으로 변환되어 가짜 데이터 G(z|y)를 생성
    
    MINIST나 CIFAR-10등의 데이터셋에 대해 학습시키는 경우 y는 레이블 정보
    일반적으로 one-hot vector를 입력으로 사용
- **Discriminator**
    실제 데이터 x와 Generator가 생성한 가짜 데이터 G(z|y)를 각각 입력으로 받음, y 정보가 각각 입력되어 진짜와 가짜를 식별함

    MINIST나 CIFAR-10 등의 데이터셋에 대해 학습시키는 경우 실제 데이터 x와 y는 알맞은 한 쌍(`"7"이라 쓰인 이미지의 경우 레이블도 7`)을 이루어야함, Generator에 입력된 y와 Discriminator에 입력되는 y는 동일한 레이블을 나타내야 함



## 13-4. 내가 원하는 숫자 이미지 만들기
### (1) Generator 구성하기

GAN과 cGAN을 간단히 구현하고 실험해보기</br>
간단한 실험을 위해 MINIST 데이터셋을 이용하고 실습 코드는 아래를 참고하였다.</br>

- [TF2-GAN](https://github.com/thisisiron/TF2-GAN)

**데이터 준비하기**
tensorflow-dataset을 사용함, 노드에는 이미 설치되어있다.</br>
```shell
# 버전 확인
$ pip list | grep tensorflow-datasets

# 라이브러리 설치하고 싶을 경우 설치 명령어
$ pip install tensorflow-datasets
```

tensorflow-datasets 라이브러리에서 간단하게 MNIST 데이터셋을 불러와 확인해 봅시다.

In [ ]:
import tensorflow_datasets as tfds

mnist, info =  tfds.load(
    "mnist", split="train", with_info=True
)

fig = tfds.show_examples(mnist, info)

```shell
Downloading and preparing dataset 11.06 MiB (download: 11.06 MiB, generated: 21.00 MiB, total: 32.06 MiB) to /aiffel/tensorflow_datasets/mnist/3.0.1...
Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]
Dataset mnist downloaded and prepared to /aiffel/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
```

![tf-dataset_minist](../images/lec13/3.png)</br>

이미지 픽셀 값을 -1 ~ 1 사이의 범위로 변경하는 함수 선언</br>
- `gan_preprocessing()`, `cgan_preprocessing()`

레이블 정보를 one-hot encoding 처리하는 함수 선언</br>
- `cgan_preprocessing()`

```text
함수를 나눈 목적

1. GAN과 cGAN을 각각 실험하기 위함
2. label 사용 유무에 따라 one-hot encoding이 적용/미적용 됨
```


In [ ]:
import tensorflow as tf

BATCH_SIZE = 128

def gan_preprocessing(data):
    image = data["image"]
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def cgan_preprocessing(data):
    image = data["image"]
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    
    label = tf.one_hot(data["label"], 10)
    return image, label

gan_datasets = mnist.map(gan_preprocessing).shuffle(1000).batch(BATCH_SIZE)
cgan_datasets = mnist.map(cgan_preprocessing).shuffle(100).batch(BATCH_SIZE)
print("✅")

```shell
✅
```

> 이미지 픽셀 값을 -1 ~ 1 사이의 범위로 변경한 이유
>> 픽셀 값 정규화; 모델의 안정성 향상될 수 있음
>> 데이터 분포의 평균을 0으로 맞추도록 함
>> 단위 분산을 갖도록 함
>> 모델의 학습 효율을 증가

In [ ]:
# 하나의 데이터셋을 이용하여 정규화가 제대로 처리되었는지를 확인
import matplotlib.pyplot as plt

for i,j in cgan_datasets : break

# 이미지 i와 라벨 j가 일치하는지 확인해 봅니다.     
print("Label :", j[0])
print("Image Min/Max :", i.numpy().min(), i.numpy().max())
plt.imshow(i.numpy()[0,...,0], plt.cm.gray)

```shell
Label : tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 1. 0.], shape=(10,), dtype=float32)
Image Min/Max : -1.0 1.0

<matplotlib.image.AxesImage at 0x7f25bdbdad00>
```

![pixel_regularization](../images/lec13/4.png)</br>

> 0 ~ 9 까지의 숫자를 one-hot 인코딩으로 표현하여 Label에 있는 리스트의 각 값들은 숫자를 의미함
> 화면에 표시된 이미지는 8, 리스트에서 9번째에 1이 표시되어야 하므로 `이미지 i와 라벨 j가 일치`함을 확인함

**GAN Generator 구성하기**
Tensorflow2의 Subclassing 방법을 이용하여 작성</br>

> Subclassing?
>> tensorflow.keras.Model을 상속받아 클래스 생성
>> 일반적으로 `__init__()` 메서드로 레이어 구성을 정의
>> 구성된 레이어를 `call()` 메서드에서 사용하여 forward propagation을 진행
>>> 이러한 방식은 PyTorch의 모델 구성 방법과도 매우 유사하여 익숙해진다면 Pytorch의 모델 구성 방법도 빠르게 습득할 수 있다.

In [ ]:
## GAN Generator 구성하기
#주석에 맞춰 Generator를 만들어주세요.

from tensorflow.keras import layers, Input, Model

class GeneratorGAN(Model):
    def __init__(self):
        super(GeneratorGAN, self).__init__()

        # 활성화함수를 'relu'를 사용하고 unit이 128인 Dense Layer를 정의해주세요,.
        self.dense_1 = tensorflow.keras.layers.Dense(128, activation='relu')
        # 활성화함수를 'relu'를 사용하고 unit이 256인 Dense Layer를 정의해주세요.
        self.dense_2 = tensorflow.keras.layers.Dense(256, activation='relu')
        # 활성화함수를 'relu'를 사용하고 unit이 512인 Dense Layer를 정의해주세요.
        self.dense_3 = tensorflow.keras.layers.Dense(512, activation='relu')
        # 활성화함수를 하이퍼볼릭 탄젠트를 사용하고 unit이 256인 Dense Layer를 정의해주세요.
        ## 입력은 256 유닛이지만 activation 처리로 인해 output은 28 * 28 * 1로 나온다
        self.dense_4 = tensorflow.keras.layers.Dense(28 * 28 * 1, activation='tanh')
        # 모양을 (28,28,1)로 변경해주세요.
        # tf.keras.layers.Reshape((28, 28, 1), input_shape(None,))
        self.reshape = tensorflow.keras.layers.Reshape((28, 28, 1))

    def call(self, noise):
        out = self.dense_1(noise)
        out = self.dense_2(out)
        out = self.dense_3(out)
        out = self.dense_4(out)
        return self.reshape(out)

print("✅")

```shell
✅
```

`__init__()` 메서드 안에서 사용할 모든 레이어를 정의함
- 4개의 fully-connected 레이어 중 하나를 제외하고 모두 ReLU 활성화 적용

`call()` 메서드에서는 노이즈를 입력받아 `__init__()`에서 정의된 레이어들을 순서대로 통과</br>
Generator는 숫자가 쓰인 이미지를 출력해야함
- 마지막 출력은 layers.Reshape()를 이용해 (28,28,1) 크기로 변환

In [ ]:
# cGAN의 Generator 생성
#주석에 맞춰 Generator를 만들어주세요.

class GeneratorCGAN(Model):
    def __init__(self):
        super(GeneratorCGAN, self).__init__()
        
        self.dense_z = layers.Dense(256, activation='relu')
        self.dense_y = layers.Dense(256, activation='relu')
        self.combined_dense = layers.Dense(512, activation='relu')
        self.final_dense = layers.Dense(28 * 28 * 1, activation='tanh')
        self.reshape = layers.Reshape((28, 28, 1))

    def call(self, noise, label):
        # 노이즈에 Dense layer를 적용시킵니다.
        noise = self.dense_z(noise)
         # 라벨에 Dense layer를 적용시킵니다.
        label = self.dense_y(label)
        # 노이즈와 라벨을 pair가 되게 합친 다음 combined_dense를 적용시킵니다. (힌트 : https://www.tensorflow.org/api_docs/python/tf/concat)
        # axis 0 : 합치는 배열 중 가장 높은 차원을 기준으로 합침
        # axis 1 : 합치는 배열 중 두 번째로 높은 차원을 기준으로 합침
        # axis -1 : 합치는 배열 중 가장 낮은 차원 뒤쪽에서부터 시작
        out = self.combined_dense(tf.concat([noise, label], axis=-1))
        # 마지막 Dense Layer를 적용시킵니다.
        out = self.final_dense(out)
        return self.reshape(out)
    
print("✅")

```shell
✅
```

cGAN의 입력은 2개(`노이즈 및 레이블 정보`)라는 점을 기억하자

## 13-5. 내가 원하는 숫자 이미지 만들기
### (2) Discriminator 구성하기

이번에는 실제 이미지와 Generator가 생성한 이미지에 대해 진짜와 가짜를 식별하는 Discriminator를 구현해보자

In [ ]:
# GAN의 Discriminator 구현
class DiscriminatorGAN(Model):
    def __init__(self):
        super(DiscriminatorGAN, self).__init__()
        self.flatten = layers.Flatten()
        
        #해당 방식은 반복문을 활용해 layer를 쌓기 때문에 좋은 테크닉중 하나입니다,.
        self.blocks = []
        for f in [512, 256, 128, 1]:
            self.blocks.append(
                layers.Dense(f, activation=None if f==1 else "relu")
            )
        
    def call(self, x):
        x = self.flatten(x)
        for block in self.blocks:
            x = block(x)
        return x
    
print("✅")

```shell
✅
```

`__init__()`에 `blocks` 리스트를 생성하고 for loop를 이용하여 필요한 레이어들을 차례로 쌓음</br>
Discriminator의 입력은 Generator가 생성한 (28,28,1) 크기의 이미지</br>
입력 이미지를 fully-connected 레이어로 학습하기 위해 `call()`에서 가장 먼저 `layers.Flatten()`을 적용
- `layers.Flatten()`으로 인해 모든 파라미터의 곱만큼의 1차원 배열이 생성된다.

[참고예제 Link](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)</br>

이어서 레이어들이 쌓여있는 `blocks`에 대해 for loop를 이용하여 레이어들을 순서대로 하나씩 꺼내 입력 데이터를 통과시킴</br>

마지막 fully-connected 레이어를 통과시키면 진짜/가짜 이미지를 나타내는 1개의 값이 출력

**cGAN Discriminator 구성하기**

cGAN의 Discriminator는 `Maxout`이라는 특별한 레이어가 사용됨</br>

>`Maxout`?
>> 두 레이어 사이를 연결할 때, 여러 개의 fully-connected 레이어를 통과시켜 그 중 가장 큰 값을 가져오도록하는 방식
>>> 2개의 fully-connected 레이어를 사용할 때의 `Maxout`을 식으로 표현하면 아래와 같다.

$$max((w_{1})^{T} + b_{1}, (w_{2})^{T}x + b_{2})$$

fully-connected 레이어를 2개만 사용하면, 다차원 공간에서 2개의 면이 교차된 모양의 activation function 처럼 작동한다고 한다.</br>

다차원 공간은 시각화가 어려워서 차원을 낮춘 1차원 fully-connected 레이어를 가정하면 아래처럼 2개의 직선으로 이루어진 activation function으로 나타낼 수 있다고 한다.</br>

![1-dim_activation_function_graph](../images/lec13/5.png)</br>

사용하는 fully-connected 레이어 갯수가 늘어난다면 점점 곡선 형태인 activation function이 될 수 있다.</br>
차원을 늘리면 다차원 공간에 곡면을 나타낼 수 있으나 시각화하기 어렵다.</br>

![smooth_activation_function_graph](../images/lec13/6.png)</br>

[Maxout에 대한 참고 논문 링크](https://arxiv.org/pdf/1302.4389.pdf)

In [ ]:
# Subclassing 형태의 Maxout 구성 코드
class Maxout(layers.Layer):
    def __init__(self, units, pieces):
        super(Maxout, self).__init__()
        self.dense = layers.Dense(units*pieces, activation="relu")
        self.dropout = layers.Dropout(.5)    
        self.reshape = layers.Reshape((-1, pieces, units))
    
    def call(self, x):
        x = self.dense(x)
        x = self.dropout(x)
        x = self.reshape(x)
        return tf.math.reduce_max(x, axis=2)

print("✅")

```shell
✅
```

`Maxout` 레이어 구성 시 `units`와 `pieces`의 설정이 필요</br>
- `units`만큼의 차원 수를 가진 fully-connected 레이어를 `pieces`개 만큼 만든다.
- 모든 레이어 중에서 최대값을 결과로 출력한다.

`Maxout` 레이어가 `units=100`, `pieces=10`으로 설정되면 다음과 같은 흐름으로 최대값이 도출된다.
- 100 차원의 representation 생성 (`units` 개수에 영향)
- representation은 총 10개 생성됨 (`pieces` 개수에 영향)
- 최종 출력되는 최대값은 1개의 100차원 짜리 representation이다.
- 식으로 나타낸다면 아래와 같다
- 위 예시에서는 각각의 `wx+b가 모두 100차원`

$$max((w_{1})^{T}x + b_{1}, (w_{2})^{T}x + b_{2}, ..., (w_{9})^{T}x + b_{9}, (w_{10})^{T}x + b_{10})$$

In [ ]:
# Maxout 레이어를 3번 사용하여 cGAN의 Discriminator 구성하기
class DiscriminatorCGAN(Model):
    def __init__(self):
        super(DiscriminatorCGAN, self).__init__()
        self.flatten = layers.Flatten()
        
        self.image_block = Maxout(240, 5)
        self.label_block = Maxout(50, 5)
        self.combine_block = Maxout(240, 4)
        
        self.dense = layers.Dense(1, activation=None)
    
    def call(self, image, label):
        image = self.flatten(image)
        image = self.image_block(image)
        label = self.label_block(label)
        x = layers.Concatenate()([image, label])
        x = self.combine_block(x)
        return self.dense(x)
    
print("✅")

```shell
✅
```

GAN의 Discriminator처럼 Generator가 생성한 (28,28,1) 크기 이미지가 입력됨</br>
입력된 이미지를 `layers.Flatten()`적용하여 1차원 값으로 변경</br>

이후 이미지 입력 및 레이블 입력 각각은 `Maxout`레이어를 한 번씩 통과</br>
`Maxout`레이어를 거친 이미지 및 레이블 입력은 결합하여 `Maxout`레이어를 한번 더 통과</br>

마지막으로 fully-connected 레이어를 통과(`self.dense(x)` 부분)하면 진짜 및 가짜 이미지를 나타내는 1개의 값을 출력

## 13-6. 내가 원하는 숫자 이미지 만들기
### (3) 학습 및 테스트하기

이전에 정의한 Generator 및 Discriminator를 이용해 MINST를 학습하고 각 모델로 직접 숫자 손글씨를 생성해보기</br>

우선 GAN, cGAN 각각의 모델 학습에 공통적으로 필요한 `loss function`과 `optimizer`를 정의</br>

진짜 및 가짜를 구별하기 위해 `Binary Cross Entropy` 사용하고, `Adam optimizer`를 이용해 학습

In [ ]:
# loss func, optimizer 정의
from tensorflow.keras import optimizers, losses

bce = losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return bce(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    return bce(tf.ones_like(real_output), real_output) + bce(tf.zeros_like(fake_output), fake_output)

gene_opt = optimizers.Adam(1e-4)
disc_opt = optimizers.Adam(1e-4)    

print("✅")

```shell
✅
```

**GAN으로 MINIST 학습하기**

이전 단계에서 구성한 GeneratorGAN, DiscriminatorGAN 모델 클래스를 이용</br>
입력 노이즈를 100차원으로 설정</br>

하나의 배치 크기 데이터로 모델을 업데이트 하는 함수를 아래와 같이 작성함

In [ ]:
# gan_generator, gan_discriminator 받아올 때 tensorflow 모듈이 없어서 충돌이 남을 확인
import tensorflow

gan_generator = GeneratorGAN()
gan_discriminator = DiscriminatorGAN()

@tf.function()
def gan_step(real_images):
    noise = tf.random.normal([real_images.shape[0], 100])
    
    with tf.GradientTape(persistent=True) as tape:
        # Generator를 이용해 가짜 이미지 생성
        fake_images = gan_generator(noise)
        # Discriminator를 이용해 진짜 및 가짜이미지를 각각 판별
        real_out = gan_discriminator(real_images)
        fake_out = gan_discriminator(fake_images)
        # 각 손실(loss)을 계산
        gene_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    # gradient 계산
    gene_grad = tape.gradient(gene_loss, gan_generator.trainable_variables)
    disc_grad = tape.gradient(disc_loss, gan_discriminator.trainable_variables)
    # 모델 학습
    gene_opt.apply_gradients(zip(gene_grad, gan_generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grad, gan_discriminator.trainable_variables))
    return gene_loss, disc_loss

print("✅")

```shell
✅
```

짧은 시간 학습된 모델 테스트</br>
100차원 노이즈 입력을 10개 사용하여 10개의 숫자 손글씨 데이터를 생성해 시각화하기

In [ ]:
EPOCHS = 10
for epoch in range(1, EPOCHS+1):
    for i, images in enumerate(gan_datasets):
        gene_loss, disc_loss = gan_step(images)

        if (i+1) % 100 == 0:
            print(f"[{epoch}/{EPOCHS} EPOCHS, {i+1} ITER] G:{gene_loss}, D:{disc_loss}")

```shell
[1/10 EPOCHS, 100 ITER] G:2.049213171005249, D:0.15589837729930878
[1/10 EPOCHS, 200 ITER] G:2.5744073390960693, D:0.13938578963279724
[1/10 EPOCHS, 300 ITER] G:2.438291549682617, D:0.16833624243736267
[1/10 EPOCHS, 400 ITER] G:2.9346017837524414, D:0.1997964084148407
[2/10 EPOCHS, 100 ITER] G:4.271846294403076, D:0.05835453420877457
[2/10 EPOCHS, 200 ITER] G:3.6204092502593994, D:0.09263713657855988
[2/10 EPOCHS, 300 ITER] G:4.199694633483887, D:0.06024268642067909
[2/10 EPOCHS, 400 ITER] G:3.4050302505493164, D:0.07571984827518463
[3/10 EPOCHS, 100 ITER] G:2.9561877250671387, D:0.1065240129828453
[3/10 EPOCHS, 200 ITER] G:4.10530948638916, D:0.12742578983306885
[3/10 EPOCHS, 300 ITER] G:5.451573371887207, D:0.04717189073562622
[3/10 EPOCHS, 400 ITER] G:5.987151145935059, D:0.09921099245548248
[4/10 EPOCHS, 100 ITER] G:4.866405487060547, D:0.02526199445128441
[4/10 EPOCHS, 200 ITER] G:3.8320398330688477, D:0.0920993834733963
[4/10 EPOCHS, 300 ITER] G:3.7405734062194824, D:0.08760032057762146
[4/10 EPOCHS, 400 ITER] G:3.327131986618042, D:0.0785658210515976
[5/10 EPOCHS, 100 ITER] G:5.966882705688477, D:0.07411172986030579
[5/10 EPOCHS, 200 ITER] G:5.448455333709717, D:0.044737137854099274
[5/10 EPOCHS, 300 ITER] G:5.872185230255127, D:0.054288603365421295
[5/10 EPOCHS, 400 ITER] G:3.4558234214782715, D:0.07067812234163284
[6/10 EPOCHS, 100 ITER] G:4.6623854637146, D:0.18453572690486908
[6/10 EPOCHS, 200 ITER] G:3.970163345336914, D:0.07567453384399414
[6/10 EPOCHS, 300 ITER] G:4.098063945770264, D:0.06308897584676743
[6/10 EPOCHS, 400 ITER] G:4.623175621032715, D:0.1211826279759407
[7/10 EPOCHS, 100 ITER] G:4.02207088470459, D:0.02991645038127899
[7/10 EPOCHS, 200 ITER] G:3.7183713912963867, D:0.0715690404176712
[7/10 EPOCHS, 300 ITER] G:5.404078960418701, D:0.15482397377490997
[7/10 EPOCHS, 400 ITER] G:4.240554332733154, D:0.0912465900182724
[8/10 EPOCHS, 100 ITER] G:2.695376396179199, D:0.08904784917831421
[8/10 EPOCHS, 200 ITER] G:5.302037715911865, D:0.04109131917357445
[8/10 EPOCHS, 300 ITER] G:4.417237758636475, D:0.019084690138697624
[8/10 EPOCHS, 400 ITER] G:5.433066368103027, D:0.0473189651966095
[9/10 EPOCHS, 100 ITER] G:4.2093095779418945, D:0.11512164026498795
[9/10 EPOCHS, 200 ITER] G:4.704627990722656, D:0.08545660227537155
[9/10 EPOCHS, 300 ITER] G:2.5713512897491455, D:0.12688413262367249
[9/10 EPOCHS, 400 ITER] G:3.9401824474334717, D:0.028414469212293625
[10/10 EPOCHS, 100 ITER] G:4.327183723449707, D:0.05319599807262421
[10/10 EPOCHS, 200 ITER] G:4.194896221160889, D:0.047951675951480865
[10/10 EPOCHS, 300 ITER] G:4.1680498123168945, D:0.12966623902320862
[10/10 EPOCHS, 400 ITER] G:6.131661415100098, D:0.02995820716023445
```

10번 학습한 결과물 확인하기

In [ ]:
import numpy as np

noise = tf.random.normal([10, 100])

output = gan_generator(noise)
output = np.squeeze(output.numpy())

plt.figure(figsize=(15,6))
for i in range(1, 11):
    plt.subplot(2,5,i)
    plt.imshow(output[i-1])

![10_epoch_result](../images/lec13/7.png)</br>

추가로 500 epoch 학습하기 위한 준비

```shell
$ mkdir -p ~/aiffel/conditional_generation/gan
$ cp ~/data/gan/GAN_500.zip ~/aiffel/conditional_generation/gan/
$ cd ~/aiffel/conditional_generation/gan && unzip GAN_500.zip
```

10번 학습한 결과물 재확인

In [ ]:
import os
weight_path = os.getenv('HOME')+'/aiffel/conditional_generation/gan/GAN_500'

noise = tf.random.normal([10, 100]) 

gan_generator = GeneratorGAN()
gan_generator.load_weights(weight_path)

output = gan_generator(noise)
output = np.squeeze(output.numpy())

plt.figure(figsize=(15,6))
for i in range(1, 11):
    plt.subplot(2,5,i)
    plt.imshow(output[i-1])

![10_epoch_result2](../images/lec13/8.png)

**cGAN으로 MNIST 학습하기**

이전 단계에서 구성한 GeneratorCGAN 및 DiscriminatorCGAN 모델 클래스 이용</br>
위에서 실행한 GAN 학습처럼 약간의 학습으로는 제대로 된 생성 결과를 얻을 수 없을 테니 연습삼아 1 epoch만 학습시키기

In [ ]:
#위에 있는 gan_step()을 참고해서 cgan_step을 완성해주세요.

cgan_generator = GeneratorCGAN()
cgan_discriminator = DiscriminatorCGAN()

@tf.function()
def cgan_step(real_images, labels):
    #[[YOUR CODE]]
    ## 100 차원의 입력 노이즈
    noise = tf.random.normal([real_images.shape[0], 100])
    
    with tf.GradientTape(persistent=True) as tape:
        #[[YOUR CODE]]
        ## Generator를 이용해 가짜 이미지 생성; cGAN의 경우 label도 필요
        fake_images = cgan_generator(noise, labels)
        
        #[[YOUR CODE]]
        ## Discriminator를 이용해 진짜 및 가짜이미지를 각각 판별
        ## cGAN의 경우 판별시 label 정보가 필요
        real_out = cgan_discriminator(real_images, labels)
        fake_out =  cgan_discriminator(fake_images, labels)
        
        #[[YOUR CODE]]
        ## 각 손실(loss)을 계산
        gene_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    
    #[[YOUR CODE]]
    ## gradient 계산
    gene_grad = tape.gradient(gene_loss, cgan_generator.trainable_variables)
    disc_grad = tape.gradient(disc_loss, cgan_discriminator.trainable_variables)
    # 모델 학습
    gene_opt.apply_gradients(zip(gene_grad, cgan_generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grad, cgan_discriminator.trainable_variables))
    return gene_loss, disc_loss


EPOCHS = 1
    
#[[YOUR CODE]]
for epoch in range(1, EPOCHS+1):
    for i, (images, labels) in enumerate(cgan_datasets):
        gene_loss, disc_loss = cgan_step(images, labels)
    
        if (i+1) % 100 == 0:
            print(f"[{epoch}/{EPOCHS} EPOCHS, {i} ITER] G:{gene_loss}, D:{disc_loss}")

```shell
[1/1 EPOCHS, 99 ITER] G:6.060835838317871, D:0.005419260822236538
[1/1 EPOCHS, 199 ITER] G:5.233320236206055, D:0.03331949561834335
[1/1 EPOCHS, 299 ITER] G:5.948786735534668, D:0.05949319526553154
[1/1 EPOCHS, 399 ITER] G:4.923242568969727, D:0.01771354302763939
```

cGAN 500번 학습을 위한 준비</br>
```shell
$ mkdir -p ~/aiffel/conditional_generation/cgan
$ cp ~/data/cgan/CGAN_500.zip ~/aiffel/conditional_generation/cgan/
$ cd ~/aiffel/conditional_generation/cgan && unzip CGAN_500.zip
```

In [ ]:
# 0 ~ 9 사이의 숫자 중 생성하길 원하는 숫자 입력 후 결과 확인하는 코드
number = 6  # TODO : 생성할 숫자를 입력해 주세요!!

weight_path = os.getenv('HOME')+'/aiffel/conditional_generation/cgan/CGAN_500'

noise = tf.random.normal([10, 100])

label = tf.one_hot(number, 10)
label = tf.expand_dims(label, axis=0)
label = tf.repeat(label, 10, axis=0)

generator = GeneratorCGAN()
generator.load_weights(weight_path)

output = generator(noise, label)
output = np.squeeze(output.numpy())

plt.figure(figsize=(15,6))
for i in range(1, 11):
    plt.subplot(2,5,i)
    plt.imshow(output[i-1])

![1_epoch_result_with_cGAN](../images/lec13/9.png)</br>

## 13-7. GAN의 입력에 이미지를 넣는다면? Pix2Pix

Pix2Pix</br>
    **이미지를 입력해서 원하는 다른 형태의 이미지로 변환**시킬 수 있는 GAN 모델
        *일반적인 GAN은 기존 노이즈 입력을 이미지로 변환함*
    
    이 구조는 최근 활발하게 연구 및 응용되는 GAN 기반의 Image-to-Image Translation 작업에서 가장 기초가 되는 연구

- [Pix2Pix Paper](https://arxiv.org/pdf/1611.07004.pdf)</br>

    Conditiaional Adversarizal Network (기존 cGAN 구조)로 **Image-to-Image Translation**을 수행

![pix2pix](../images/lec13/10.png)</br>

위 결과의 첫 번째 (**Labels to Street Scene**) 이미지는</br>
픽셀 별로 레이블 정보만 존재하는 segmentation map을 입력, </br>
실제 거리 사진을 생성함.</br>

기존 cGAN은 입력(noise, label) 이후 fully-connected 레이어를 연속적으로 쌓아 만듦</br>
Pix2Pix는 *이미지 변환이 목적*이기 때문에 convolution 레이어를 활용함</br>
    GAN 구조 기반이므로 Generator와 Discriminator를 가진다.

**Pix2Pix (Generator)**

## 13-8. 난 스케치를 할 테니 너는 채색을 하거라
### (1) 데이터 준비하기

pix2pix 모델에 대해 직접 구현하고 실험해보기</br>
사용해볼 데이터셋은 `Sketch2Pokemon`</br>

- [Sketch2Pokemon - Kaggle](https://www.kaggle.com/datasets/norod78/sketch2pokemon)

아래 명령어를 사용하여 데이터 준비</br>

```shell
$ mkdir -p ~/aiffel/conditional_generation/data
$ ln -s ~/data/sketch2pokemon.zip ~/aiffel/conditional_generation/data
$ cd ~/aiffel/conditional_generation/data && unzip sketch2pokemon.zip
```

In [ ]:
# Sketch2Pokemon 데이터셋 확인
import os

data_path = os.getenv('HOME')+'/aiffel/conditional_generation/data/pokemon_pix2pix_dataset/train/'
print("number of train examples :", len(os.listdir(data_path)))

```shell
number of train examples : 830
```

In [ ]:
# 각각의 이미지 확인
## 학습용 데이터셋에서 임의로 6장을 선택하여 시각화
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(20,15))
for i in range(1, 7):
    f = data_path + os.listdir(data_path)[np.random.randint(800)]
    img = cv2.imread(f, cv2.IMREAD_COLOR)
    plt.subplot(3,2,i)
    plt.imshow(img)

![pokemon_6_images](../images/lec13/11.png)

In [ ]:
# 하나의 이미지 열어서 크기 확인해보기
f = data_path + os.listdir(data_path)[0]
img = cv2.imread(f, cv2.IMREAD_COLOR)
print(img.shape)

```shell
(256, 512, 3)
```

모델 학습에 사용할 데이터를 (256, 256, 3) 크기의 2개 이미지로 분할하여 사용하면 될 것 같다.</br>
 - (256, 256, 3) 이미지는 각각 스케치본, 색칠본

In [ ]:
## 이미지 나누기
import tensorflow as tf

def normalize(x):
    x = tf.cast(x, tf.float32)
    return (x/127.5) - 1

def denormalize(x):
    x = (x+1)*127.5
    x = x.numpy()
    return x.astype(np.uint8)

def load_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, 3)
    
    w = tf.shape(img)[1] // 2
    sketch = img[:, :w, :] 
    sketch = tf.cast(sketch, tf.float32)
    colored = img[:, w:, :] 
    colored = tf.cast(colored, tf.float32)
    return normalize(sketch), normalize(colored)

f = data_path + os.listdir(data_path)[1]
sketch, colored = load_img(f)

plt.figure(figsize=(10,7))
plt.subplot(1,2,1); plt.imshow(denormalize(sketch))
plt.subplot(1,2,2); plt.imshow(denormalize(colored))

```shell
<matplotlib.image.AxesImage at 0x7eff5c7c5c10>
```

![pokemon_divide_2_imgs](../images/12.png)</br>

Augmentation?
    - 기존 데이터를 변형, 확장 혹은 재구성하는 방법
        - 위 작업을 수행 시 데이터셋의 크기는 증가한다
    - 모델의 일반화 능력을 향상시킴
    - overfitting 방지, 데이터의 불균형 문제 해소를 위해 사용
    - 기법 종류
        - 상/하/좌/우 이미지 뒤집기
        - 회전
        - 자르기 (Cropping)
        - 확대 및 축소


In [ ]:
## Augmentation 적용을 위한 함수 선언
from tensorflow import image
from tensorflow.keras.preprocessing.image import random_rotation

@tf.function() # 빠른 텐서플로 연산을 위해 @tf.function()을 사용합니다. 
def apply_augmentation(sketch, colored):
    stacked = tf.concat([sketch, colored], axis=-1)
    
    _pad = tf.constant([[30,30],[30,30],[0,0]])
    if tf.random.uniform(()) < .5:
        padded = tf.pad(stacked, _pad, "REFLECT")
    else:
        padded = tf.pad(stacked, _pad, "CONSTANT", constant_values=1.)

    out = image.random_crop(padded, size=[256, 256, 6])
    
    out = image.random_flip_left_right(out)
    out = image.random_flip_up_down(out)
    
    if tf.random.uniform(()) < .5:
        degree = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32)
        out = image.rot90(out, k=degree)
    
    return out[...,:3], out[...,3:]   

print("✅")

```shell
✅
```

`apply_augmentation()`은 스케치 및 채색된 2개 이미지를 입력으로 받음</br>
여러 가지 연산을 두 이미지에 동일하게 적용

여러가지 연산
    1. 두 이미지가 채널 축으로 연결 (`tf.concat`)
        - 각 이미지가 3채널인 경우 합쳐진 이후(concatenated) 6채널이 됨
    2. 연결된 이미지에 각 50% 확률로 `Refection padding` 또는 `constant padding`이 30 pixel 만큼 적용됨 (`tf.pad`)
    3. padding이 적용된 이미지에서 (256, 256, 6) 크기를 가진 이미지를 임의로 잘라냄 (`tf.image.random_crop`)
    4. 잘라낸 이미지를 50% 확률로 가로로 뒤집음 (`tf.image.random_flip_left_right`)
    5. 가로로 뒤집힌 이미지를 50% 확률로 세로로 뒤집음 (`tf.image.random_flip_up_down`)
    6. 세로로 뒤집힌 이미지를 50% 확률로 회전시킴 (`tf.image.rot90`)

In [ ]:
# apply_augmentation() 함수에 데이터셋을 적용해 시각화해보기
plt.figure(figsize=(15,13))
img_n = 1
for i in range(1, 13, 2):
    augmented_sketch, augmented_colored = apply_augmentation(sketch, colored)
    
    plt.subplot(3,4,i)
    plt.imshow(denormalize(augmented_sketch)); plt.title(f"Image {img_n}")
    plt.subplot(3,4,i+1); 
    plt.imshow(denormalize(augmented_colored)); plt.title(f"Image {img_n}")
    img_n += 1

![apply_augemntation()_result](../images/lec13/13.png)

In [ ]:
# apply_augmentation()의 과정들을 학습 데이터에 적용하여, 잘 적용되었는지 확인
# 하나의 이미지만 시각화하여 확인하는 코드
from tensorflow import data

def get_train(img_path):
    sketch, colored = load_img(img_path)
    sketch, colored = apply_augmentation(sketch, colored)
    return sketch, colored

train_images = data.Dataset.list_files(data_path + "*.jpg")
train_images = train_images.map(get_train).shuffle(100).batch(4)

sample = train_images.take(1)
sample = list(sample.as_numpy_iterator())
sketch, colored = (sample[0][0]+1)*127.5, (sample[0][1]+1)*127.5

plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.imshow(sketch[0].astype(np.uint8))
plt.subplot(1,2,2); plt.imshow(colored[0].astype(np.uint8))

```shell
<matplotlib.image.AxesImage at 0x7efe79bf9a60>
```

![apply_augmentation()_with_model_train](../images/lec13/14.png)

## 13-9. 난 스케치를 할 테니 너는 채색을 하거라
### (2) Generator 구성하기

**Generator의 구성요소 알아보기**
pix2pix 논문에서 표기한 encoder의 "C64"의 의미
    - 하이퍼파라미터를 가진 레이어들의 조합
        - 64개의 4x4 필터에 stride 2를 적용한 Convolution Layer
            - `0.2`의 slope를 가지는 LeakyReLU
            - Batch Normalization은 첫 번째 C64 Layer의 encoder에 적용되어 있지 않다.


논문의 `6.1 Network arhitectures`와 `6.1.1 Generator architectures`를 확인하자.</br>

pix2pix 논문에서 표기한 decoder의 "CD512"의 의미
    - 512개의 4x4 필터를 가진 (Transposed) Convolution Layer
        - stride 2 적용
    - 이후 Batch Normalization
    - 이후 50% 데이터에 대해 Dropout
    - 이후 ReLU 사용
        


**Generator 구현하기**

논문에서 언급한 것(`C64`, `C128` 등)과 같이 `Convolution -> BatchNora -> LeakyReLU`</br>
의 3개 레이어로 구성된 기본적인 블록을 하나의 레이어로 만들기

In [ ]:
from tensorflow.keras import layers, Input, Model

class EncodeBlock(layers.Layer):
    def __init__(self, n_filters, use_bn=True):
        super(EncodeBlock, self).__init__()
        self.use_bn = use_bn       
        self.conv = layers.Conv2D(n_filters, 4, 2, "same", use_bias=False)
        self.batchnorm = layers.BatchNormalization()
        self.lrelu= layers.LeakyReLU(0.2)

    def call(self, x):
        x = self.conv(x)
        if self.use_bn:
            x = self.batchnorm(x)
        return self.lrelu(x)

print("✅")

```shell
✅
```

`__init__()` 메서드에서 `n_filters`, `use_bn`을 설정하여 사용할 필터의 개수와 BatchNorm 사용여부를 결정할 수 있다.</br>



In [ ]:
## EncodeBlock을 여러 번 가져다 사용하여 Encooder 구성하기
class Encoder(layers.Layer):
    def __init__(self):
        super(Encoder, self).__init__()
        filters = [64,128,256,512,512,512,512,512]
        
        # for문과 리스트 filter를 활용해서 EncoderBlock을 쌓아주세요.
        # 조건 1. 첫번째 EncoderBlock의 경우 Batch Normalization을 생략해주세요.
        self.blocks = []

        for i, f in enumerate(filters):
            if i == 0:
                self.blocks.append(EncodeBlock(f, use_bn=False))
            else:
                self.blocks.append(EncodeBlock(f))
    
    def call(self, x):
        for block in self.blocks:
            x = block(x)
        return x
    
    def get_summary(self, input_shape=(256,256,3)):
        inputs = Input(input_shape)
        return Model(inputs, self.call(inputs)).summary()

print("✅")

```shell
✅
```

`filters`: 각 블록을 거치면서 사용할 필터의 개수를 저장하는 리스트</br>
`blocks`: 사용할 블록들을 정의해둔 리스트</br>
`call()`: 차례대로 블록들을 통과하는 메서드
    - Encoder의 첫 번째 블록에서는 BatchNorm을 사용하지 않음
`get_summary`: 레이어가 제대로 구성되었는지를 확인하기 위한 용도

In [ ]:
# Encoder에 (256, 256, 3) 크기의 데이터가 입력되었을때,
# 어떤 크기의 데이터가 출력되는지를 확인
Encoder().get_summary()

```shell
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
encode_block (EncodeBlock)   (None, 128, 128, 64)      3072      
_________________________________________________________________
encode_block_1 (EncodeBlock) (None, 64, 64, 128)       131584    
_________________________________________________________________
encode_block_2 (EncodeBlock) (None, 32, 32, 256)       525312    
_________________________________________________________________
encode_block_3 (EncodeBlock) (None, 16, 16, 512)       2099200   
_________________________________________________________________
encode_block_4 (EncodeBlock) (None, 8, 8, 512)         4196352   
_________________________________________________________________
encode_block_5 (EncodeBlock) (None, 4, 4, 512)         4196352   
_________________________________________________________________
encode_block_6 (EncodeBlock) (None, 2, 2, 512)         4196352   
_________________________________________________________________
encode_block_7 (EncodeBlock) (None, 1, 1, 512)         4196352   
=================================================================
Total params: 19,544,576
Trainable params: 19,538,688
Non-trainable params: 5,888
_________________________________________________________________
```

In [ ]:
# Decoder 구성하기
## 사용할 기본 블록을 정의
###이후 이 블록을 여러번 반복하는 클래스
class DecodeBlock(layers.Layer):
    def __init__(self, f, dropout=True):
        super(DecodeBlock, self).__init__()
        self.dropout = dropout
        self.Transconv = layers.Conv2DTranspose(f, 4, 2, "same", use_bias=False)
        self.batchnorm = layers.BatchNormalization()
        self.relu = layers.ReLU()
        
    def call(self, x):
        x = self.Transconv(x)
        x = self.batchnorm(x)
        if self.dropout:
            x = layers.Dropout(.5)(x)
        return self.relu(x)

    
class Decoder(layers.Layer):
    def __init__(self):
        super(Decoder, self).__init__()
        filters = [512,512,512,512,256,128,64]
        # for문을 이용해서 모델을 쌓아주세요.
        # 조건 1. 3번째 block까지는 Dropout을 사용하되 그 이후에는 Dropout을 사용하지 마세요.
        # for문이 끝난 다음 Conv2DTranspose를 쌓아주되 output 차원수는 3, filter 사이즈는 4, stride는 2로 구성해주시고 자동 패딩 적용해주시되 bias는 사용하지 않습니다.
        self.blocks = []
        for i, f in enumerate(filters):
            if i < 3:
                self.blocks.append(DecodeBlock(f))
            else:
                self.blocks.append(DecodeBlock(f, dropout=False))
        
    def call(self, x):
        for block in self.blocks:
            x = block(x)
        return x
            
    def get_summary(self, input_shape=(1,1,512)):
        inputs = Input(input_shape)
        return Model(inputs, self.call(inputs)).summary()
        
print("✅")

```shell
✅
```

(1, 1, 512) 크기에 데이터가 입력되었을 때, 어떤 크기가 출력되는지 확인해보기

In [ ]:
Decoder().get_summary()

```shell
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_3 (InputLayer)         [(None, 1, 1, 512)]       0         
_________________________________________________________________
decode_block (DecodeBlock)   (None, 2, 2, 512)         4196352   
_________________________________________________________________
decode_block_1 (DecodeBlock) (None, 4, 4, 512)         4196352   
_________________________________________________________________
decode_block_2 (DecodeBlock) (None, 8, 8, 512)         4196352   
_________________________________________________________________
decode_block_3 (DecodeBlock) (None, 16, 16, 512)       4196352   
_________________________________________________________________
decode_block_4 (DecodeBlock) (None, 32, 32, 256)       2098176   
_________________________________________________________________
decode_block_5 (DecodeBlock) (None, 64, 64, 128)       524800    
_________________________________________________________________
decode_block_6 (DecodeBlock) (None, 128, 128, 64)      131328    
=================================================================
Total params: 19,539,712
Trainable params: 19,534,720
Non-trainable params: 4,992
_________________________________________________________________
```

위에서 구성한 Encoder와 Decoder를 연결시키는 코드를 구성해보자</br>
`tf.keras.Model`을 상속받아서 Encoder와 Decoder를 연결하는 Generator를 구성해보자

In [ ]:
class EncoderDecoderGenerator(Model):
    def __init__(self):
        super(EncoderDecoderGenerator, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    
    def call(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
   
    def get_summary(self, input_shape=(256,256,3)):
        inputs = Input(input_shape)
        return Model(inputs, self.call(inputs)).summary()
        

EncoderDecoderGenerator().get_summary()

```shell
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
encoder_1 (Encoder)          (None, 1, 1, 512)         19544576  
_________________________________________________________________
decoder_2 (Decoder)          (None, 128, 128, 64)      19539712  
=================================================================
Total params: 39,084,288
Trainable params: 39,073,408
Non-trainable params: 10,880
_________________________________________________________________
```

Generator를 잘 작동시키기 위해서는 약 4,000만 개의 파라미터를 잘 학습시켜야 함

## 13-10. 난 스케치를 할 테니 너는 채색을 하거라
### (3) Generator 재구성하기

Encoder와 Decoder에 사용되는 기본적인 블록은 아래와 같이 구현하였다.

In [ ]:
class EncodeBlock(layers.Layer):
    def __init__(self, n_filters, use_bn=True):
        super(EncodeBlock, self).__init__()
        self.use_bn = use_bn       
        self.conv = layers.Conv2D(n_filters, 4, 2, "same", use_bias=False)
        self.batchnorm = layers.BatchNormalization()
        self.lrelu = layers.LeakyReLU(0.2)

    def call(self, x):
        x = self.conv(x)
        if self.use_bn:
            x = self.batchnorm(x)
        return self.lrelu(x)

    
class DecodeBlock(layers.Layer):
    def __init__(self, f, dropout=True):
        super(DecodeBlock, self).__init__()
        self.dropout = dropout
        self.Transconv = layers.Conv2DTranspose(f, 4, 2, "same", use_bias=False)
        self.batchnorm = layers.BatchNormalization()
        self.relu = layers.ReLU()
        
    def call(self, x):
        x = self.Transconv(x)
        x = self.batchnorm(x)
        if self.dropout:
            x = layers.Dropout(.5)(x)
        return self.relu(x)
    
print("✅")

```shell
✅
```

정의된 블록들을 이용하여 **U-Net Generator**를 정의하기</br>

이전 구현에는 없었던 skip connection이 `call()` 내부에서 어떻게 구현되었는지 잘 확인해보기

In [ ]:
class UNetGenerator(Model):
    def __init__(self):
        super(UNetGenerator, self).__init__()
        encode_filters = [64,128,256,512,512,512,512,512]
        decode_filters = [512,512,512,512,256,128,64]
        
        self.encode_blocks = []
        for i, f in enumerate(encode_filters):
            if i == 0:
                self.encode_blocks.append(EncodeBlock(f, use_bn=False))
            else:
                self.encode_blocks.append(EncodeBlock(f))
        
        self.decode_blocks = []
        for i, f in enumerate(decode_filters):
            if i < 3:
                self.decode_blocks.append(DecodeBlock(f))
            else:
                self.decode_blocks.append(DecodeBlock(f, dropout=False))
        
        self.last_conv = layers.Conv2DTranspose(3, 4, 2, "same", use_bias=False)
    
    def call(self, x):
        features = []
        for block in self.encode_blocks:
            x = block(x)
            features.append(x)
        
        features = features[:-1]
                    
        for block, feat in zip(self.decode_blocks, features[::-1]):
            x = block(x)
            x = layers.Concatenate()([x, feat])
        
        x = self.last_conv(x)
        return x
                
    def get_summary(self, input_shape=(256,256,3)):
        inputs = Input(input_shape)
        return Model(inputs, self.call(inputs)).summary()

print("✅")

```shell
✅
```

Encoder와 Decoder 사이의 skip connection을 위해 `features`라는 리스트를 만듦</br>
이 리스트는 Encoder 내에서 사용된 각 블록들의 출력을 차례대로 담음</br>

이후 Decoder에서는 Encoder의 최종 출력이 입력으로 들어옴</br>
Decoder 단계가 진행될 떄, `features` 리스트에 있는 각각의 출력들이 *Decoder 블록 연산 후 함께 연결*</br>
연결된 값은 다음 블록의 입력으로 사용된다.

```text
[문답 정리]

Q. 위 코드의 call() 내에서 features = features[:-1] 는 왜 필요할까요?
A. Encoder의 마지막 출력(feature 리스트 마지막 항목)은 Decoder로 직접 입력되므로 skip connection 대상이 아니다.

Q. 위 코드의 call() 내의 Decoder 연산 부분에서 features[::-1] 는 왜 필요할까요?
A. features에 Encoder의 블록별 출력이 쌓여있고(PUSH) Decoder에서 차례대로 사용(POP)하기 위해서 연산함.
```

완성된 U-Net 구조 Generator 내부의 각 출력이 적절한지 아래 코드로 확인

In [ ]:
UNetGenerator().get_summary()

```shell
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
encode_block_16 (EncodeBlock)   (None, 128, 128, 64) 3072        input_5[0][0]                    
__________________________________________________________________________________________________
encode_block_17 (EncodeBlock)   (None, 64, 64, 128)  131584      encode_block_16[0][0]            
__________________________________________________________________________________________________
encode_block_18 (EncodeBlock)   (None, 32, 32, 256)  525312      encode_block_17[0][0]            
__________________________________________________________________________________________________
encode_block_19 (EncodeBlock)   (None, 16, 16, 512)  2099200     encode_block_18[0][0]            
__________________________________________________________________________________________________
encode_block_20 (EncodeBlock)   (None, 8, 8, 512)    4196352     encode_block_19[0][0]            
__________________________________________________________________________________________________
encode_block_21 (EncodeBlock)   (None, 4, 4, 512)    4196352     encode_block_20[0][0]            
__________________________________________________________________________________________________
encode_block_22 (EncodeBlock)   (None, 2, 2, 512)    4196352     encode_block_21[0][0]            
__________________________________________________________________________________________________
encode_block_23 (EncodeBlock)   (None, 1, 1, 512)    4196352     encode_block_22[0][0]            
__________________________________________________________________________________________________
decode_block_14 (DecodeBlock)   (None, 2, 2, 512)    4196352     encode_block_23[0][0]            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2, 2, 1024)   0           decode_block_14[0][0]            
                                                                 encode_block_22[0][0]            
__________________________________________________________________________________________________
decode_block_15 (DecodeBlock)   (None, 4, 4, 512)    8390656     concatenate[0][0]                
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 4, 4, 1024)   0           decode_block_15[0][0]            
                                                                 encode_block_21[0][0]            
__________________________________________________________________________________________________
decode_block_16 (DecodeBlock)   (None, 8, 8, 512)    8390656     concatenate_1[0][0]              
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 8, 8, 1024)   0           decode_block_16[0][0]            
                                                                 encode_block_20[0][0]            
__________________________________________________________________________________________________
decode_block_17 (DecodeBlock)   (None, 16, 16, 512)  8390656     concatenate_2[0][0]              
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 16, 16, 1024) 0           decode_block_17[0][0]            
                                                                 encode_block_19[0][0]            
__________________________________________________________________________________________________
decode_block_18 (DecodeBlock)   (None, 32, 32, 256)  4195328     concatenate_3[0][0]              
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 32, 32, 512)  0           decode_block_18[0][0]            
                                                                 encode_block_18[0][0]            
__________________________________________________________________________________________________
decode_block_19 (DecodeBlock)   (None, 64, 64, 128)  1049088     concatenate_4[0][0]              
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 64, 64, 256)  0           decode_block_19[0][0]            
                                                                 encode_block_17[0][0]            
__________________________________________________________________________________________________
decode_block_20 (DecodeBlock)   (None, 128, 128, 64) 262400      concatenate_5[0][0]              
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 128, 128, 128 0           decode_block_20[0][0]            
                                                                 encode_block_16[0][0]            
__________________________________________________________________________________________________
conv2d_transpose_21 (Conv2DTran (None, 256, 256, 3)  6144        concatenate_6[0][0]              
==================================================================================================
Total params: 54,425,856
Trainable params: 54,414,976
Non-trainable params: 10,880
__________________________________________________________________________________________________
```

Encoder-Decoder Generator 구조에서 학습해야할 파라미터 수 : 약 4천만개</br>
Skip connection 추가한 U-Net Generator에서 학습해야할 파라미터 수 : 약 5500만 개</br>

    - U-Net Generator의 Decoder 구조 내 파라미터가 증가함

> U-Net Generator에서 사용한 skip-connection으로 인해 Decoder의 각 블록에서 입력받는 채널 수가 증가했고, 이에 따라 블록 내 convolution 레이어에서 사용하는 필터 크기가 증가하여 학습해야할 파라미터가 증가함

## 13-11. 난 스케치를 할 테니 너는 채색을 하거라
### (4) Discriminator 구성하기

**Discriminator 구현하기**
Discriminator에 사용할 기본 블록 만들기

In [ ]:
class DiscBlock(layers.Layer):
    def __init__(self, n_filters, stride=2, custom_pad=False, use_bn=True, act=True):
        super(DiscBlock, self).__init__()
        self.custom_pad = custom_pad
        self.use_bn = use_bn
        self.act = act
        
        if custom_pad:
            self.padding = layers.ZeroPadding2D()
            self.conv = layers.Conv2D(n_filters, 4, stride, "valid", use_bias=False)
        else:
            self.conv = layers.Conv2D(n_filters, 4, stride, "same", use_bias=False)
        
        self.batchnorm = layers.BatchNormalization() if use_bn else None
        self.lrelu = layers.LeakyReLU(0.2) if act else None
        
    def call(self, x):
        if self.custom_pad:
            x = self.padding(x)
            x = self.conv(x)
        else:
            x = self.conv(x)
                
        if self.use_bn:
            x = self.batchnorm(x)
            
        if self.act:
            x = self.lrelu(x)
        return x 

print("✅")

```shell
✅
```

`__init__()`에서 다양한 설정</br>

`n_filters`: 필터의 수</br>
`stride`: 필터가 순회하는 간격</br>
`custom_pad`: 출력 feature map 크기 조절할 수 있는 패딩 설정</br>
`use_bn`: BatchNorm의 사용 여부</br>
`act`: 활성화 함수 사용 여부</br>


노드 질문 / 답 정리
```text
Q. 위에서 만든 DiscBlock의 설정을 다음과 같이 하여 DiscBlock(n_filters=64, stride=1, custom_pad=True, use_bn=True, act=True) 으로 생성된 블록에 (width, height, channel) = (128, 128, 32) 크기가 입력된다면, 블록 내부에서 순서대로 어떠한 레이어를 통과하는지, 그리고 각 레이어를 통과했을 때 출력 크기는 어떻게 되는지 적어주세요.

A. 패딩 레이어 통과 layers.ZeroPadding2D() → (130,130,32)
Convolution 레이어 통과 layers.Conv2D(64,4,1,"valid") → (127,127,64)
BatchNormalization 레이어 통과 layers.BatchNormalization() → (127,127,64)
LeakyReLU 활성화 레이어 통과 layers.LeakyReLU(0.2) → (127,127,64)
```

위 과정을 확인하기 위해 간단히 코드를 작성해보자.

In [ ]:
inputs = Input((128,128,32))
out = layers.ZeroPadding2D()(inputs)
out = layers.Conv2D(64, 4, 1, "valid", use_bias=False)(out)
out = layers.BatchNormalization()(out)
out = layers.LeakyReLU(0.2)(out)

Model(inputs, out).summary()

```shell
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_6 (InputLayer)         [(None, 128, 128, 32)]    0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 130, 130, 32)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 127, 127, 64)      32768     
_________________________________________________________________
batch_normalization_45 (Batc (None, 127, 127, 64)      256       
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 127, 127, 64)      0         
=================================================================
Total params: 33,024
Trainable params: 32,896
Non-trainable params: 128
_________________________________________________________________
```

Pix2Pix의 Discriminator가 70x70 PatchGAN을 사용한다.</br>
이 때문에 최종 출력을 (30,30) 크기로 맞추어야 한다.</br>

이를 이용해 바로 Discriminator를 만들어보자.

In [ ]:
class Discriminator(Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        filters = [64,128,256,512,1]
        self.blocks = [layers.Concatenate()]
        # For문을 활용해서 DiscBlock을 쌓아주세요.

        # DiscBlock에 필요한 arguments 초기값 설정
        n_filters, stride = None, None  # type setting as int, int
        custom_pad, use_bn, act = None, None, None # type setting as Bool, Bool, Bool

        for idx_of_filters in range(len(filters)):
            # 조건 1 : 3번째까지 stride는 2로 주되 이후에는 1로 주세요
            # 조건 2 : 3번째까지 custom padding을 주지 않아도 되는데 이후에는 주세요.
            if idx_of_filters < 3:
                stride = 2
                custom_pad = False
            else:
                stride = 1
                custom_pad = True
            # 조건 3: 1번째와 5번째에서는 Batch Normalization을 사용하지 마세요.    
            if idx_of_filters == 0 or idx_of_filters == 4:
                use_bn = False
            else:
                use_bn = True
            # 조건 4 : 1번째부터 4번째까지 LeakyReLU를 적용하고 마지막에는 sigmoid를 적용하세요.
            if idx_of_filters <= 3:
                act = True
            else:
                act = False
            
            # flters 순서대로 n_filters의 값을 적용; 조건과 무관함
            n_filters = filters[idx_of_filters]

            # DiscBlock() appending; self.blocks의 길이를 늘리자
            self.blocks.append(DiscBlock(n_filters=n_filters, stride=stride, 
                    custom_pad=custom_pad, use_bn=use_bn, act=act))

        ## (sigmoid의 경우 따로 정의해야 합니다)
        self.sigmoid = layers.Activation("sigmoid")
    
    def call(self, x, y):        
        #[[YOUR CODE]]
        # Block 통과한 출력값 out 초기화
        out = None
        # __init__()에서 쌓은 blocks를 local var로 가져와보자
        blocks = self.blocks

        # list인 blocks를 순회
        for idx_of_blocks in range(len(blocks)):
            if idx_of_blocks == 0:
                # blocks[0] == layers.Concatenate()
                out = blocks[idx_of_blocks]([x, y])
            else:
                out = blocks[idx_of_blocks](out)
        
        return self.sigmoid(out)
    
    def get_summary(self, x_shape=(256,256,3), y_shape=(256,256,3)):
        x, y = Input(x_shape), Input(y_shape) 
        return Model((x, y), self.call(x, y)).summary()
    
print("✅")

```shell
✅
```

이전 노드에서는 6개 블록을 각각 따로 만들었지만, 이번에는 for loop를 사용하여 만들어보았다.</br>

각 블록의 출력 크기가 알맞게 되었는지 아래 코드로 확인해보자.

In [ ]:
Discriminator().get_summary()

```shell
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_9 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 256, 256, 6)  0           input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
disc_block_5 (DiscBlock)        (None, 128, 128, 64) 6144        concatenate_8[0][0]              
__________________________________________________________________________________________________
disc_block_6 (DiscBlock)        (None, 64, 64, 128)  131584      disc_block_5[0][0]               
__________________________________________________________________________________________________
disc_block_7 (DiscBlock)        (None, 32, 32, 256)  525312      disc_block_6[0][0]               
__________________________________________________________________________________________________
disc_block_8 (DiscBlock)        (None, 31, 31, 512)  2099200     disc_block_7[0][0]               
__________________________________________________________________________________________________
disc_block_9 (DiscBlock)        (None, 30, 30, 1)    8192        disc_block_8[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 30, 30, 1)    0           disc_block_9[0][0]               
==================================================================================================
Total params: 2,770,432
Trainable params: 2,768,640
Non-trainable params: 1,792
__________________________________________________________________________________________________
```

두 개의 `(256,256,3)` 크기 입력으로 최종 `(30,30,1)` 출력을 만들었고, 아래의 Discriminator를 나타낸 그림과 각 출력 크기가 일치함을 확인</br>

![discriminator_structure](../images/lec13/15.png)</br>

시험 삼아 임의의 (256, 256, 3) 크기의 입력과 (30, 30) 출력을 시각화해본다.

In [ ]:
x = tf.random.normal([1,256,256,3])
y = tf.random.uniform([1,256,256,3])

disc_out = Discriminator()(x, y)
plt.imshow(disc_out[0, ... ,0])
plt.colorbar()

```shell
<matplotlib.colorbar.Colorbar at 0x7efe79d092b0>
```

![256_square_rgbimg_input_30_square_output](../images/lec13/16.png)</br>

이전 PatchGAN에 대해 설명했던 것처럼, </br>
위 (30,30) 크기의 결과 이미지의 각 픽셀 값은 원래 입력 이미지의 패치 분류 결과이다.</br>
- 패치 사이즈는 (70,70), 원래 입력 이미지 크기는 (256,256)

각각의 픽셀 값은 `sigmoid` 함수의 결과값이므로 0 ~ 1 사이의 값을 가지고,</br>
이 값은 진짜/가짜 데이터를 판별하는데 사용한다.

## 13-12. 난 스케치를 할 테니 너는 채색을 하거라
### (5) 학습 및 테스트하기

구현된 Generator와 Discriminator를 학습시켜보고,</br>
스케치를 입력하여 채색된 이미지를 생성해보자.</br>

학습에 필요한 손실 함수부터 정의해야한다.</br>
아래 그림은 논문의 여러 실험 결과 중 손실 함수 선택에 따른 결과 차이 예시이다.</br>

![loss_function_test_result](../images/lec13/17.png)</br>

레이블 정보만 있는 입력에 대해 여러 손실 함수를 사용해 실제 이미지를 만들어 낸 결과는, 일반적인 GAN의 손실 함수에 L1을 추가로 이용했을 때 가장 실제에 가까운 이미지를 생성해 냈습니다. 이번 실험에서도 두 가지 손실 함수를 모두 사용해 봅시다.

In [ ]:
## 두 가지 손실함수 정의
from tensorflow.keras import losses

bce = losses.BinaryCrossentropy(from_logits=False)
mae = losses.MeanAbsoluteError()

def get_gene_loss(fake_output, real_output, fake_disc):
    l1_loss = mae(real_output, fake_output)
    gene_loss = bce(tf.ones_like(fake_disc), fake_disc)
    return gene_loss, l1_loss

def get_disc_loss(fake_disc, real_disc):
    return bce(tf.zeros_like(fake_disc), fake_disc) + bce(tf.ones_like(real_disc), real_disc)

print("✅")

```shell
✅
```

- Generator의 손실함수 (위 코드의 `get_gene_loss()`)는 총 3개의 입력이 있다.
    - `fake_disc`: Generator가 생성한 가짜 이미지를 Discriminator에 입력해 얻어진 값
    - `tf.ones_like()`: 실제 이미지를 뜻하는 "1"(`one`)과 비교하기 위해 사용
    - `fake_output`: 생성한 가짜 이미지
    - `real_output`: 실제 이미지
    - MAE (Mean Absolute Error): L1 손실을 계산하기 위해 `fake_output`과 `real_output` 사이의 오차값

사용할 optimizer는 논문과 동일하게 아래와 같이 설정하자.

In [ ]:
from tensorflow.keras import optimizers

gene_opt = optimizers.Adam(2e-4, beta_1=.5, beta_2=.999)
disc_opt = optimizers.Adam(2e-4, beta_1=.5, beta_2=.999)

print("✅")

```shell
✅
```

하나의 배치를 입력했을 때, 가중치 1회 업데이트 하는 과정을 구현한 코드는 아래와 같다.

In [ ]:
@tf.function
def train_step(sketch, real_colored):
    with tf.GradientTape() as gene_tape, tf.GradientTape() as disc_tape:
    # 이전에 배웠던 내용을 토대로 train_step을 구성해주세요.
        # Generator 예측
        fake_colored = generator(sketch, training=True)
        # Discriminator 예측
        fake_disc = discriminator(sketch, fake_colored, training=True)
        real_disc = discriminator(sketch, real_colored, training=True)
        # Generator 손실 계산
        gene_loss, l1_loss = get_gene_loss(fake_colored, real_colored, fake_disc)
        gene_total_loss = gene_loss + (100 * l1_loss) # 논문 기준 L1 Loss값에 λ값을 곱하는 수식이 있음; λ = 100
        # Discriminator 손실 계산
        disc_loss = get_disc_loss(fake_disc, real_disc)
    
    gene_gradient = gene_tape.gradient(gene_total_loss, generator.trainable_variables)
    disc_gradient = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gene_opt.apply_gradients(zip(gene_gradient, generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_gradient, discriminator.trainable_variables))
    return gene_loss, l1_loss, disc_loss

print("✅")

```shell
✅
```

논문에서 Generator의 손실을 아래와 같이 정의하였다.</br>

$$G^{*} = arg\frac minG \frac minD L_{cGAN}(G,D) + \lambda L_{L1}(G)$$

위 식에서 `λ`는 학습 과정에서 L1 손실이 얼마나 반영되는지를 나타낸다.</br>
논문에서 `λ = 100`을 사용하였기에 코드에 100으로 적용하였다.

앞서 정의한 함수를 이용해서 학습 진행. (10 epoch)

In [ ]:
EPOCHS = 10

generator = UNetGenerator()
discriminator = Discriminator()

for epoch in range(1, EPOCHS+1):
    for i, (sketch, colored) in enumerate(train_images):
        g_loss, l1_loss, d_loss = train_step(sketch, colored)
                
        # 10회 반복마다 손실을 출력합니다.
        if (i+1) % 10 == 0:
            print(f"EPOCH[{epoch}] - STEP[{i+1}] \
                    \nGenerator_loss:{g_loss.numpy():.4f} \
                    \nL1_loss:{l1_loss.numpy():.4f} \
                    \nDiscriminator_loss:{d_loss.numpy():.4f}", end="\n\n")

```shell
# 결과에 warning이 보기 싫다면 warnings 모듈을 적용해주자. 이곳에서는 생략함
EPOCH[1] - STEP[10]                     
Generator_loss:1.0736                     
L1_loss:0.5112                     
Discriminator_loss:1.0302

EPOCH[1] - STEP[20]                     
Generator_loss:1.5368                     
L1_loss:0.3022                     
Discriminator_loss:1.7035

...

EPOCH[1] - STEP[190]                     
Generator_loss:2.1154                     
L1_loss:0.2574                     
Discriminator_loss:0.7090

EPOCH[1] - STEP[200]                     
Generator_loss:1.0449                     
L1_loss:0.2074                     
Discriminator_loss:0.6052

...

EPOCH[10] - STEP[10]                     
Generator_loss:1.6309                     
L1_loss:0.2465                     
Discriminator_loss:0.9081

EPOCH[10] - STEP[20]                     
Generator_loss:2.5589                     
L1_loss:0.2322                     
Discriminator_loss:0.2783

...

EPOCH[10] - STEP[190]                     
Generator_loss:3.1080                     
L1_loss:0.2567                     
Discriminator_loss:0.8310

EPOCH[10] - STEP[200]                     
Generator_loss:1.3011                     
L1_loss:0.2895                     
Discriminator_loss:0.4816
```

정확한 시간 추산은 못했지만 최소 5분은 걸리는 느낌이다</br>

아래 코드로 학습해본 모델에 채색 수행하기

In [ ]:
test_ind = 1

f = data_path + os.listdir(data_path)[test_ind]
sketch, colored = load_img(f)

pred = generator(tf.expand_dims(sketch, 0))
pred = denormalize(pred)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1); plt.imshow(denormalize(sketch))
plt.subplot(1,3,2); plt.imshow(pred[0])
plt.subplot(1,3,3); plt.imshow(denormalize(colored))

```shell
<matplotlib.image.AxesImage at 0x7eff5c8c68e0>
```

![gen_10_epoch_result](../images/lec13/18.png)</br>

Kaggle 테스트 내역으로 Pix2Pix의 128 epoch 학습 후 테스트 결과가 아래와 같다고 한다.</br>

![kaggle_128_epoch_test_case](../images/lec13/19.png)</br>